In [1]:
import pandas as pd
from readFiles import fullRead
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder

from tabpfn import TabPFNClassifier

OSError: [WinError 127] No se encontró el proceso especificado. Error loading "c:\ProgramData\Anaconda3\lib\site-packages\torch\lib\caffe2_detectron_ops.dll" or one of its dependencies.

In [ ]:
def predictSex(pathToTable):
    df = fullRead(pathToTable)
    enc = OrdinalEncoder()
    enc.fit(df[["Sweetener", "Time"]])
    df[["Sweetener", "Time"]] = enc.transform(df[["Sweetener", "Time"]])
    X, y = df.drop(["numVol", "Sex"], axis=1), df["Sex"]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

    classifier = TabPFNClassifier(device='cpu', N_ensemble_configurations=32)

    classifier.fit(X_train, y_train)
    y_eval, p_eval = classifier.predict(X_test, return_winning_probability=True)

    print('Accuracy in sex clasiffication', pathToTable, accuracy_score(y_test, y_eval))

def predictSweet(pathToTable):
    df = fullRead(pathToTable, anthro=True)
    df = df[df["Time"] == "Final"]
    enc = OrdinalEncoder()
    enc.fit(df[["Sex"]])
    df[["Sex"]] = enc.transform(df[["Sex"]])
    X, y = df.drop(["numVol", "Sweetener", "Time"], axis=1), df["Sweetener"]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

    classifier = TabPFNClassifier(device='cpu', N_ensemble_configurations=32)

    classifier.fit(X_train, y_train)
    y_eval, p_eval = classifier.predict(X_test, return_winning_probability=True)

    print('Accuracy in sweetener clasiffication', pathToTable, accuracy_score(y_test, y_eval))    

In [ ]:
predictSex("data/urineFlav_ord.csv")
predictSweet("data/urineFlav_ord.csv")
predictSex("data/urineAnt_ord.csv")
predictSweet("data/urineAnt_ord.csv")
predictSex("data/plasmFlav_ord.csv")
predictSweet("data/plasmFlav_ord.csv")
predictSex("data/plasmAnt_ord.csv")
predictSweet("data/plasmAnt_ord.csv")

Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Accuracy in sex clasiffication data/urineFlav_ord.csv 0.6024096385542169
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Accuracy in sweetener clasiffication data/urineFlav_ord.csv 0.5476190476190477
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Accuracy in sex clasiffication data/urineAnt_ord.csv 0.6746987951807228
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Accuracy in sweetener clasiffication data/urineAnt_ord.csv 0.47619047619047616
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Accuracy in sex clasiffication data/plasmFlav_ord.csv 0.4878048780487805
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Accuracy in sweetener clasiffication data/plasmFlav_ord.csv 